In [1]:
import pandas as pd

In [2]:
import json
import xmltodict

In [3]:
with open('Raw_store_Details.xml') as xml_file:
    data_dict = xmltodict.parse(xml_file.read())

In [4]:
orders_dict=data_dict['s:Envelope']['s:Body']['GetOrdersResponse']['GetOrdersResult']['Orders']['Order']

In [5]:
orders_df = pd.DataFrame(orders_dict)

In [6]:
def getClosedTime(row):
    return orders_dict[row.name]['ClosedTime']['a:DateTime']
def getCustomerId(row):
    return orders_dict[row.name]['CustomerId']['@i:nil']
def getTaxExemptId(row):
    return orders_dict[row.name]['CustomerId']['@i:nil']
def getOrderId(row):
    return row.name+1

In [7]:
orders_df['ClosedTime']=orders_df.apply(getClosedTime,axis=1)
orders_df['CustomerId']=orders_df.apply(getCustomerId,axis=1)
orders_df['TaxExemptId']=orders_df.apply(getTaxExemptId,axis=1)
orders_df['OrderId']=orders_df.apply(getOrderId,axis=1)

In [8]:
orders_df.columns

Index(['BusinessDate', 'ClosedTime', 'Count', 'CustomerId', 'Deposits',
       'DestinationId', 'Discounts', 'DisplaySubtotal', 'DisplayTax',
       'EmployeeId', 'Entries', 'FirstSendTime', 'GrossSales', 'GuestCount',
       'GuestCountFractional', 'Id', 'IdEncoded', 'IdNoLocation', 'IsClosed',
       'IsFutureOrder', 'IsRefund', 'IsTaxExempt', 'ModifiedTime', 'Name',
       'NetSales', 'Number', 'OpenedTime', 'OrderAdditionalDetails',
       'Payments', 'PickupTime', 'Promotions', 'Rounding', 'SectionId',
       'Subtotal', 'Surcharges', 'Tax', 'TaxExemptId', 'TerminalId',
       'TillNumber', 'Total', 'OrderId'],
      dtype='object')

In [15]:
order_items_df=pd.DataFrame()

In [16]:
for i, a in enumerate(orders_df['Entries']):
    if(type(a)==type({})):
        for b in a['OrderEntry']:
            if(type(b)==type({})):
                try:
                    entry_df=pd.DataFrame(b)
                    entry_df['OrderId']=i+1
                    order_items_df=pd.concat([order_items_df, entry_df],ignore_index=True)
                except e:
                    print(e)

In [17]:
first_column = order_items_df.pop('OrderId')
order_items_df.insert(0, 'OrderId', first_column)
first_column = order_items_df.pop('OrderId')
order_items_df.insert(0, 'OrderId', first_column)

first_column = orders_df.pop('OrderId')
orders_df.insert(0, 'OrderId', first_column)
first_column = orders_df.pop('OrderId')
orders_df.insert(0, 'OrderId', first_column)

In [18]:
orders_df.head()

,OrderId,BusinessDate,ClosedTime,Count,CustomerId,Deposits,DestinationId,Discounts,DisplaySubtotal,DisplayTax,...,Promotions,Rounding,SectionId,Subtotal,Surcharges,Tax,TaxExemptId,TerminalId,TillNumber,Total
0,1,2022-09-30T00:00:00,2022-09-30T15:21:14.0092452Z,1,true,None,1,None,59.98,4.51,...,None,0.0,1,59.98,None,4.51,true,2,65538,64.49
1,2,2022-09-30T00:00:00,2022-09-30T15:07:45.479309Z,1,true,None,4,None,14.56,1.10,...,None,0.0,1,14.56,None,1.1,true,207830,65537,15.66
2,3,2022-09-30T00:00:00,2022-09-30T15:29:55.4450749Z,1,true,None,4,None,3.99,0.30,...,None,0.0,1,3.99,None,0.3,true,207830,65537,4.29
3,4,2022-09-30T00:00:00,2022-09-30T15:36:28.3747222Z,1,true,None,4,None,15.46,1.16,...,None,0.0,1,15.46,None,1.16,true,207830,65537,16.62
4,5,2022-09-30T00:00:00,2022-09-30T15:46:40.1217087Z,1,true,None,4,None,18.37,1.38,...,None,0.0,1,18.37,None,1.38,true,207830,65537,19.75


In [19]:
order_items_df.head()

,OrderId,@i:type,DayPartId,Description,Id,ItemId,Price,Promotions,CompositeOrderItemId,Denominator,...,IsDeleted,IsVoided,ItemGrossSales,ItemNetSales,Modifiers,NetSales,Note,SplitItemId,Taxes,Void
0,1,OrderItem,3,"BLIZZARD CAKE - 8""",1,270146,29.99,None,true,1,...,false,false,29.99,29.99,NaN,29.99,None,0,NaN,true
1,1,OrderItem,3,"BLIZZARD CAKE - 8""",1,270146,29.99,None,NaN,1,...,false,false,29.99,29.99,"{'DisplayPrice': '0.0', 'GrossSales': '0.0', '...",29.99,None,0,NaN,NaN
2,1,OrderItem,3,"BLIZZARD CAKE - 8""",1,270146,29.99,None,NaN,1,...,false,false,29.99,29.99,NaN,29.99,None,0,"[{'Amount': '0.0', 'Id': '1', 'IsInclusive': '...",NaN
3,1,OrderItem,3,"BLIZZARD CAKE - 8""",3,270146,29.99,None,true,1,...,false,false,29.99,29.99,NaN,29.99,None,0,NaN,true
4,1,OrderItem,3,"BLIZZARD CAKE - 8""",3,270146,29.99,None,NaN,1,...,false,false,29.99,29.99,"{'DisplayPrice': '0.0', 'GrossSales': '0.0', '...",29.99,None,0,NaN,NaN
